<a href="https://colab.research.google.com/github/zhou671/STRAVE/blob/master/style-transfer-for-videos-tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clone Our GitHub Repo

In [19]:
%cd /content/
!git clone https://github.com/zhou671/STRAVE.git

/content
Cloning into 'STRAVE'...
remote: Enumerating objects: 174, done.
remote: Counting objects: 100% (174/174), done.
remote: Compressing objects: 100% (126/126), done.
remote: Total 174 (delta 89), reused 131 (delta 46), pack-reused 0
Receiving objects: 100% (174/174), 2.54 MiB | 11.24 MiB/s, done.
Resolving deltas: 100% (89/89), done.


# Install DeepMatching GPU Version
[DeepMatching](https://thoth.inrialpes.fr/src/deepmatching/) is a matching algorithm developped by Jerome Revaud in 2013. Its purpose is to compute dense correspondences between two images. In this work, we will use DeepMatching together with [DeepFlow](https://thoth.inrialpes.fr/src/deepflow) to calculate the optic flow of image pairs. When using the CPU version, on average it takes 35 seconds per image pair. To save your time, we provide the download link to all optic flow files of the sample video in the `demo` folder. If you need the GPU version of DeepMatching to speed up, then please follow these instructions.

## Use Our Pre-Compiled Files (Recommended)
For your convenience, we have compiled all files needed with Colab and tested on both Colab CPU and GPU.

## Build From Scratch
If Colab has updated the package versions and our pre-compiled files don't work, you may want to compile yourself. We'll leave necessary comments on how to modify files of an older version. So don't panic.

You can test the correctness of the following steps with either CPU or GPU backend. This is because even if you have no access to NVIDIA driver (`nvidia-smi`) when using CPU, CUDA is still there in the file system.

#### Change CUDA Runtime Version
Because the .cu files of DeepMatching used `cusparseScsrmm2`, which was removed since CUDA 11.0, we need to compile these files with previous version. We found that CUDA 10.1 would cause weird errors, but **CUDA 10.0** worked well. You can check CUDA runtime version by `!nvcc -V`.

In [1]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


The default CUDA runtime version should be 11.0. However, the GPU version of DeepMatching was released far before CUDA 11.0. We need to switch to an older CUDA version to compile CUDA files.

In [1]:
!which nvcc

/usr/local/cuda/bin/nvcc


In [2]:
!ls -ll /usr/local/ | grep cuda

lrwxrwxrwx  1 root root    9 Apr  7 13:29 cuda -> cuda-11.0
drwxr-xr-x 16 root root 4096 Apr  7 13:24 cuda-10.0
drwxr-xr-x 15 root root 4096 Apr  7 13:26 cuda-10.1
drwxr-xr-x  1 root root 4096 Apr  7 13:28 cuda-11.0


`/usr/local/cuda` is simply a soft link, so we can make it link to the CUDA folder we want.

In [10]:
!rm /usr/local/cuda
!ln -s /usr/local/cuda-10.0  /usr/local/cuda

Now we successfully link `/usr/local/cuda` to `/usr/local/cuda-11.0`. We will switch back to CUDA 11.0 later to run TensorFlow 2.x.

#### Compile Caffe V1.0
It seems that it has been about 2 years since the last commit to the GitHub [repo](https://github.com/BVLC/caffe). Among all the versions, version 1.0 works well. However, DeepMatching included an `.hpp` file that was removed and reconstructed since Caffe version `rc3`, so we need to modify these files when compiling DeepMatching later.

We will use CUDA 10.0 and Python 2.7 to compile Caffe. We use Python 2.7 because DeepMatching GPU currently only supports Python 2.x.

First install some dependencies. This may take 3-7 minutes.



In [11]:
# References:
# https://github.com/mhmdghazal/Install-Caffe-using-Colab-/blob/master/Caffe_Work.ipynb
# http://caffe.berkeleyvision.org/install_apt.html
# http://caffe.berkeleyvision.org/installation.html#compilation
with open('/etc/apt/sources.list') as f:
  txt = f.read()
with open('/etc/apt/sources.list', 'w') as f:
  f.write(txt.replace('# deb-src','deb-src'))
!apt update
!apt upgrade

!sudo apt build-dep caffe-cuda

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://archive.canonical.com/ubuntu bionic InRelease [10.2 kB]
Ign:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.canonical.com/ubuntu bionic/partner Sources [1,229 B]
Get:13 http://archi

Next, clone the repo.

In [12]:
%cd /content/
!git clone https://github.com/BVLC/caffe.git
%cd /content/caffe
!git tag -l  # show all tags (versions)
!git checkout tags/1.0  # use version 1.0 (the latest version)
!!cp Makefile.config.example Makefile.config

/content
Cloning into 'caffe'...
remote: Enumerating objects: 65274, done.
remote: Total 65274 (delta 0), reused 0 (delta 0), pack-reused 65274
Receiving objects: 100% (65274/65274), 74.14 MiB | 22.77 MiB/s, done.
Resolving deltas: 100% (41249/41249), done.
/content/caffe
1.0
acm-mm-oss
bsd
rc
rc2
rc3
rc4
rc5
rcnn-release
v0.1
v0.9
v0.99
v0.999
v0.9999
Note: checking out 'tags/1.0'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at eeebdab1 Merge pull request #5530 from willyd/nccl-py3


[]

Before we compile Caffe, we need to modify `Makefile.config`.
We made the following changes.


*   Uncomment line 12-14. Otherwise, there are errors. (Seems that you need to insall OpenCV.)
*   Specify CUDA_DIR in line 31. Remember that we have changed the soft link `/usr/local/cuda`. But to prevent any issues, we carefully specify the folder name with CUDA version (`cuda-10.0`).
*   Modify CUDA_ARCH in line 40. With a free Colab GPU, you may meet K80, P4, and T4 (use `!nvidia-smi` to check the GPU version). If you have access to other GPUs, refer to this article [Matching CUDA arch and CUDA gencode for various NVIDIA architectures](https://arnon.dk/matching-sm-architectures-arch-and-gencode-for-various-nvidia-cards/)
for more information.
*   Modify PYTHON_INCLUDE in line 74. You can use `!python2 -c "import numpy; print(numpy.get_include())"` to find the correct path.
*   Add hdf5 files in line 99-100. You can use `!find /usr -name "hdf5"` to find the include and lib path. Refer to this blog [Caffe fatal error: hdf5.h: No such file or directory error solution](https://programmerah.com/caffe-fatal-error-hdf5-h-no-such-file-or-directory-error-solution-6298/) for more information.

Run the following cell to rewrite `Makefile.config`.



In [13]:
%%writefile /content/caffe/Makefile.config
## Refer to http://caffe.berkeleyvision.org/installation.html
# Contributions simplifying and improving our build system are welcome!

# cuDNN acceleration switch (uncomment to build with cuDNN).
# USE_CUDNN := 1

# CPU-only switch (uncomment to build without GPU support).
# CPU_ONLY := 1

# uncomment to disable IO dependencies and corresponding data layers
USE_OPENCV := 0
USE_LEVELDB := 0
USE_LMDB := 0
# This code is taken from https://github.com/sh1r0/caffe-android-lib
# USE_HDF5 := 0

# uncomment to allow MDB_NOLOCK when reading LMDB files (only if necessary)
#	You should not set this flag if you will be reading LMDBs with any
#	possibility of simultaneous read and write
# ALLOW_LMDB_NOLOCK := 1

# Uncomment if you're using OpenCV 3
# OPENCV_VERSION := 3

# To customize your choice of compiler, uncomment and set the following.
# N.B. the default for Linux is g++ and the default for OSX is clang++
# CUSTOM_CXX := g++

# CUDA directory contains bin/ and lib/ directories that we need.
CUDA_DIR := /usr/local/cuda-10.0
# On Ubuntu 14.04, if cuda tools are installed via
# "sudo apt-get install nvidia-cuda-toolkit" then use this instead:
# CUDA_DIR := /usr

# CUDA architecture setting: going with all of them.
# For CUDA < 6.0, comment the *_50 through *_61 lines for compatibility.
# For CUDA < 8.0, comment the *_60 and *_61 lines for compatibility.
# For CUDA >= 9.0, comment the *_20 and *_21 lines for compatibility.
CUDA_ARCH := #-gencode arch=compute_20,code=sm_20 \
		#-gencode arch=compute_20,code=sm_21 \
		#-gencode arch=compute_30,code=sm_30 \
		-gencode arch=compute_35,code=sm_35 \
		-gencode arch=compute_50,code=sm_50 \
		-gencode arch=compute_52,code=sm_52 \
		-gencode arch=compute_60,code=sm_60 \
		-gencode arch=compute_61,code=sm_61 \
		-gencode arch=compute_61,code=compute_61 \
        -gencode arch=compute_75,code=[sm_75,compute_75]

# BLAS choice:
# atlas for ATLAS (default)
# mkl for MKL
# open for OpenBlas
BLAS := atlas
# Custom (MKL/ATLAS/OpenBLAS) include and lib directories.
# Leave commented to accept the defaults for your choice of BLAS
# (which should work)!
# BLAS_INCLUDE := /path/to/your/blas
# BLAS_LIB := /path/to/your/blas

# Homebrew puts openblas in a directory that is not on the standard search path
# BLAS_INCLUDE := $(shell brew --prefix openblas)/include
# BLAS_LIB := $(shell brew --prefix openblas)/lib

# This is required only if you will compile the matlab interface.
# MATLAB directory should contain the mex binary in /bin.
# MATLAB_DIR := /usr/local
# MATLAB_DIR := /Applications/MATLAB_R2012b.app

# NOTE: this is required only if you will compile the python interface.
# We need to be able to find Python.h and numpy/arrayobject.h.
PYTHON_INCLUDE := /usr/include/python2.7 \
		/usr/local/lib/python2.7/dist-packages/numpy/core/include
# Anaconda Python distribution is quite popular. Include path:
# Verify anaconda location, sometimes it's in root.
# ANACONDA_HOME := $(HOME)/anaconda
# PYTHON_INCLUDE := $(ANACONDA_HOME)/include \
		# $(ANACONDA_HOME)/include/python2.7 \
		# $(ANACONDA_HOME)/lib/python2.7/site-packages/numpy/core/include

# Uncomment to use Python 3 (default is Python 2)
# PYTHON_LIBRARIES := boost_python3 python3.5m
# PYTHON_INCLUDE := /usr/include/python3.5m \
#                 /usr/lib/python3.5/dist-packages/numpy/core/include

# We need to be able to find libpythonX.X.so or .dylib.
PYTHON_LIB := /usr/lib
# PYTHON_LIB := $(ANACONDA_HOME)/lib

# Homebrew installs numpy in a non standard path (keg only)
# PYTHON_INCLUDE += $(dir $(shell python -c 'import numpy.core; print(numpy.core.__file__)'))/include
# PYTHON_LIB += $(shell brew --prefix numpy)/lib

# Uncomment to support layers written in Python (will link against Python libs)
# WITH_PYTHON_LAYER := 1

# Whatever else you find you need goes here.
INCLUDE_DIRS := $(PYTHON_INCLUDE) /usr/local/include /usr/include/hdf5/serial
LIBRARY_DIRS := $(PYTHON_LIB) /usr/local/lib /usr/lib /usr/lib/x86_64-linux-gnu/hdf5/serial

# If Homebrew is installed at a non standard location (for example your home directory) and you use it for general dependencies
# INCLUDE_DIRS += $(shell brew --prefix)/include
# LIBRARY_DIRS += $(shell brew --prefix)/lib

# NCCL acceleration switch (uncomment to build with NCCL)
# https://github.com/NVIDIA/nccl (last tested version: v1.2.3-1+cuda8.0)
# USE_NCCL := 1

# Uncomment to use `pkg-config` to specify OpenCV library paths.
# (Usually not necessary -- OpenCV libraries are normally installed in one of the above $LIBRARY_DIRS.)
# USE_PKG_CONFIG := 1

# N.B. both build and distribute dirs are cleared on `make clean`
BUILD_DIR := build
DISTRIBUTE_DIR := distribute

# Uncomment for debugging. Does not work on OSX due to https://github.com/BVLC/caffe/issues/171
# DEBUG := 1

# The ID of the GPU that 'make runtest' will use to run unit tests.
TEST_GPUID := 0

# enable pretty build (comment to see full commands)
Q ?= @


Overwriting /content/caffe/Makefile.config


All set! Now let's compile Caffe. The whole process may take 6-8 minutes.

In [14]:
# 6-7 minutes
%cd /content/caffe/
!sudo make all -j 2  # 2 processes

/content/caffe
CXX src/caffe/blob.cpp
PROTOC src/caffe/proto/caffe.proto
CXX src/caffe/solvers/adam_solver.cpp
CXX src/caffe/solvers/sgd_solver.cpp
CXX src/caffe/solvers/rmsprop_solver.cpp
CXX src/caffe/solvers/nesterov_solver.cpp
CXX src/caffe/solvers/adagrad_solver.cpp
CXX src/caffe/solvers/adadelta_solver.cpp
CXX src/caffe/parallel.cpp
CXX src/caffe/syncedmem.cpp
CXX src/caffe/data_transformer.cpp
CXX src/caffe/layer.cpp
CXX src/caffe/util/upgrade_proto.cpp
CXX src/caffe/util/hdf5.cpp
CXX src/caffe/util/benchmark.cpp
CXX src/caffe/util/math_functions.cpp
CXX src/caffe/util/signal_handler.cpp
CXX src/caffe/util/db_leveldb.cpp
CXX src/caffe/util/cudnn.cpp
CXX src/caffe/util/io.cpp
CXX src/caffe/util/insert_splits.cpp
CXX src/caffe/util/db.cpp
CXX src/caffe/util/db_lmdb.cpp
CXX src/caffe/util/im2col.cpp
CXX src/caffe/util/blocking_queue.cpp
CXX src/caffe/common.cpp
CXX src/caffe/net.cpp
CXX src/caffe/layer_factory.cpp
CXX src/caffe/layers/batch_norm_layer.cpp
CXX src/caffe/layers/tanh_

Now you should see `build` and `.build_release`. `.build_release` is hidden in the menu bar. However, you can call `!ls -a /content/caffe/` to see hidden files.

In [17]:
!ls -a /content/caffe/

.		 CONTRIBUTORS.md  .github		   matlab
..		 data		  .gitignore		   models
build		 distribute	  include		   python
.build_release	 docker		  INSTALL.md		   README.md
caffe.cloc	 docs		  LICENSE		   scripts
cmake		 .Doxyfile	  Makefile		   src
CMakeLists.txt	 examples	  Makefile.config	   tools
CONTRIBUTING.md  .git		  Makefile.config.example  .travis.yml


In [18]:
!ls -a /content/caffe/ | grep build

build
.build_release


#### Compile DeepMatching GPU V1.0

You can download the source file [here](https://thoth.inrialpes.fr/src/deepmatching/code/deepmatching_gpu_1.0.zip). 
However, we modified a lot to accomodate to Caffe V1.0.
So in this tutorial, we'll use our modified source file in `/content/STRAVE/opticFlow/web_gpudm_1.0_source`. Refer to `opticFlow/web_gpudm_1.0_source/README.txt` for more information.

In [20]:
!cp /content/STRAVE/opticFlow/web_gpudm_1.0_source /content/web_gpudm_1.0

Then, add a softlink named `caffe` in the `web_gpudm_1.0` directory that points toward the `caffe` directory.

In [21]:
!ln -s /content/caffe /content/web_gpudm_1.0/caffe

Next, install some packages we will need when compiling DeepMatching.

You will need `swig` to automatically generate some scripts.

If you use GPU backend, you will need `libgoogle-glog-dev`, or you will see the following error when running DeepMatching.
```
GPU ImportError: libglog.so.0: cannot open shared object file: No such file or directory
```

In [22]:
!sudo apt-get install swig
!sudo apt-get install libgoogle-glog-dev  # When using GPU backend

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  linux-headers-4.15.0-140 linux-headers-4.15.0-140-generic r-cran-covr
  r-cran-crosstalk r-cran-dt r-cran-htmlwidgets r-cran-later r-cran-lazyeval
  r-cran-promises r-cran-rex
Use 'sudo apt autoremove' to remove them.
The following additional packages will be installed:
  swig3.0
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig swig3.0
0 upgraded, 2 newly installed, 0 to remove and 4 not upgraded.
Need to get 1,100 kB of archives.
After this operation, 5,822 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig3.0 amd64 3.0.12-1 [1,094 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig amd64 3.0.12-1 [6,460 B]
Fetched 1,100 kB in 1s (1,160 kB/s)
debconf: unable t

You also need to let the system see libcaffe.so.1.0.0.

You may see this message after running `!ldconfig`, but it doesn't matter.
```
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link
```

In [24]:
!echo /content/caffe/.build_release/lib > /etc/ld.so.conf.d/caffe.conf
!ldconfig

/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link



Now let's start compiling DeepMatching!

Because DeepMatching only supports Python 2, and `python` may be called during compiling, we first let `python` point to Python 2.

In [72]:
!mv /usr/local/bin/python /usr/local/bin/python3
!ln -s /usr/bin/python2.7 /usr/local/bin/python
!python --version

Python 2.7.17


We need to make twice.

In the first time, you will see many warnings and errors. Don't panic! It will take 1 minute.

It's expected that you see this error message. Now let's leave it behind and move on to the second make.

```
Makefile:50: recipe for target 'gpudm_wrap.cxx' failed
make: *** [gpudm_wrap.cxx] Error 202
make: *** Waiting for unfinished jobs....
```

In [62]:
%cd /content/web_gpudm_1.0/
!make cleanswig
!sudo make all -j2

/content/web_gpudm_1.0
rm -f *.pyc *~ _gpudm.so gpudm_wrap.o ./extra_layers.o ./my_im2col.o ./extra_layers.cuo ./my_im2col.cuo
rm -f gpudm.py gpudm_wrap.cxx gpudm_wrap.o
swig -python -c++ -I/usr/include/python2.7 -Icaffe/include -I/usr/local/cuda-10.0/include -Igoogle_tools/include/ -I/usr/lib/x86_64-linux-gnu/atlas/include -Icaffe/include -Icaffe/.build_release/src -I/usr/include/hdf5/serial -I/usr/local/lib/python2.7/dist-packages/numpy/core/include gpudm.swig
g++ -g -O2 -c extra_layers.cpp -fPIC -I/usr/include/python2.7 -Icaffe/include -I/usr/local/cuda-10.0/include -Igoogle_tools/include/ -I/usr/lib/x86_64-linux-gnu/atlas/include -Icaffe/include -Icaffe/.build_release/src -I/usr/include/hdf5/serial -I/usr/local/lib/python2.7/dist-packages/numpy/core/include -Lcaffe -o extra_layers.o
caffe/include/caffe/common.hpp:31: Error: Unknown namespace 'google'
caffe/include/caffe/common.hpp:120: Warning 362: operator= ignored
caffe/include/caffe/common.hpp:125: Warning 325: Nested class not 

It takes another 1 minute to finish the second make.

In [63]:
%cd /content/web_gpudm_1.0/
!sudo make -j2

/content/web_gpudm_1.0
g++ -g -O2 -c gpudm_wrap.cxx -fPIC -I/usr/include/python2.7 -Icaffe/include -I/usr/local/cuda-10.0/include -Igoogle_tools/include/ -I/usr/lib/x86_64-linux-gnu/atlas/include -Icaffe/include -Icaffe/.build_release/src -I/usr/include/hdf5/serial -I/usr/local/lib/python2.7/dist-packages/numpy/core/include -o gpudm_wrap.o
/usr/local/cuda-10.0/bin/nvcc -gencode arch=compute_35,code=sm_35 -gencode arch=compute_50,code=sm_50 -gencode arch=compute_52,code=sm_52 -gencode arch=compute_60,code=sm_60 -gencode arch=compute_61,code=sm_61 -gencode arch=compute_61,code=compute_61 -gencode arch=compute_75,code=[sm_75,compute_75] -Xcompiler -fPIC -I/usr/include/python2.7 -Icaffe/include -I/usr/local/cuda-10.0/include -Igoogle_tools/include/ -I/usr/lib/x86_64-linux-gnu/atlas/include -Icaffe/include -Icaffe/.build_release/src -I/usr/include/hdf5/serial -I/usr/local/lib/python2.7/dist-packages/numpy/core/include -g -O2 -c my_im2col.cu -o my_im2col.cuo
g++ -g -O2 -fPIC -I/usr/include/p

Don't forget to recover `python` to 3.x.

In [73]:
!mv /usr/local/bin/python3 /usr/local/bin/python
!python --version

Python 3.7.10


Now let's test if DeepMatching works.

`gpudm.py` is automatically generated. There is a small error to fix. In line 14543, change `child_grid!=None` to `child_grid is not None`. Otherwise, you will see
```
ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
```

In [66]:
with open('/content/web_gpudm_1.0/gpudm.py') as f:
  txt = f.read()
with open('/content/web_gpudm_1.0/gpudm.py', 'w') as f:
  f.write(txt.replace('child_grid!=None','child_grid is not None'))

Now let's run an example. Remenber to use `python2`.

For GPU, simply add `-GPU` argument.

See `/content/web_gpudm_1.0/README.txt` for more information.

You should get the following output: 
```
36 36 36 26 3.77558 13
36 44 36 34 3.65536 24
[...]
28 4 22 6 3.59854 94
28 20 28 10 3.75238 90
28 28 28 18 3.77126 83
```

In [67]:
%cd /content/web_gpudm_1.0/
# CPU command
!python2 deep_matching_gpu.py liberty1.png liberty2.png  
# GPU command
# !python2 deep_matching_gpu.py liberty1.png liberty2.png -GPU  

/content/web_gpudm_1.0
E0412 19:52:28.814328 22372 common.cpp:114] Cannot create Cublas handle. Cublas won't be available.
E0412 19:52:28.824152 22372 common.cpp:121] Cannot create Curand generator. Curand won't be available.
found 49 correspondences for img pair 0
36 36 36 26 3.77558 0
36 44 36 34 3.65536 0
36 12 38 2 3.71165 0
36 52 42 52 3.69999 0
36 60 42 62 3.69605 0
36 20 36 10 3.77609 0
36 28 36 18 3.78784 0
44 36 44 30 3.78537 0
44 44 46 38 3.68796 0
44 12 44 0 3.71276 0
44 52 48 48 3.71118 0
44 60 50 58 3.67123 0
44 28 44 20 3.76802 0
12 36 10 42 3.72974 0
12 44 10 52 3.69926 0
12 12 8 18 3.68889 0
12 4 6 10 3.6482 0
12 20 8 26 3.66893 0
12 28 10 36 3.6736 0
52 36 52 26 3.78852 0
52 44 54 36 3.7071 0
52 12 52 0 3.73858 0
52 52 56 46 3.69766 0
52 60 58 54 3.64645 0
52 20 50 10 3.75464 0
52 28 52 16 3.76556 0
60 36 60 24 3.79248 0
60 44 62 34 3.62088 0
60 12 58 0 3.7401 0
60 20 58 8 3.76444 0
60 28 60 16 3.79014 0
4 36 2 44 3.74037 0
4 44 2 52 3.71631 0
4 12 0 20 3.6906 0
4 52 1